# 18.06 Problem Set 9

## Problem 1 [5+5+5+5]
### (a)

In [1]:
M = [ 0.899  0.5  0.1  0.5  0
      0.1    0.3  0.4  0.0  0
      0.0    0.1  0.4  0.0  0
      0.001  0.0  0.1  0.49 0
      0.0    0.1  0.0  0.01 1 ]

using LinearAlgebra
λ, X = eigen(M);

# check something about λ?

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
5-element Vector{ComplexF64}:
 0.10181769041350378 + 0.0im
  0.5003800408562611 - 0.04926978942324787im
  0.5003800408562611 + 0.04926978942324787im
  0.9864222278739748 + 0.0im
                 1.0 + 0.0im
vectors:
5×5 Matrix{ComplexF64}:
  -0.515533+0.0im   -0.736042-0.0im        …    0.637631+0.0im  0.0+0.0im
   0.805065+0.0im     0.15488-0.265381im        0.103141+0.0im  0.0+0.0im
  -0.269991+0.0im    0.228909-0.15202im        0.0175882+0.0im  0.0+0.0im
  0.0708807+0.0im    0.386141+0.368314im      0.00482745+0.0im  0.0+0.0im
 -0.0904219+0.0im  -0.0338875+0.0490864im      -0.763188+0.0im  1.0+0.0im

All $|\mathrm{Re}(\lambda)|$ must be less than $1$ or $\mathrm{Re}(\lambda)$ equal to $1$,

### (b)
Let's assume the eigenvectors and eigenvalues of $M$ are $x_i$ and $\lambda_i$, we can find some 
$$e_1 = c_1x_1 + c_2x_2 + c_3x_3 + c_4x_4 + c_5x_5$$
W.L.O.G let $1 = \lambda_1 \geq \mathrm{Re}(\lambda_2) \geq \mathrm{Re}(\lambda_3) \geq \mathrm{Re}(\lambda_4) \geq \mathrm{Re}(\lambda_5)$.  
Using the property of eigenvectors, 
$$M^ne_1 = c_1\lambda_1^nx_1 + c_2\lambda_2^nx_2 + c_3\lambda_3^nx_3 + c_4\lambda_4^nx_4 + c_5\lambda_5^nx_5$$
Therefore, 
$$(M^n - M^{n-1})e_1 = c_2(\lambda_2 - 1)\lambda_2^{n-1}x_2 + c_3(\lambda_3 - 1)\lambda_3^{n-1}x_3 + c_4(\lambda_4 - 1)\lambda_4^{n-1}x_4 + c_5(\lambda_5 - 1)\lambda_5^{n-1}x_5$$
When $n$ is sufficiently large, only the ones (except $1$) with largest real part matters, therefore
$$(M^n - M^{n-1})e_1 \approx c\lambda_2^n = (\text{some constant})e^{n \ln \lambda_2}$$
where $c$ is some constant.  

In [2]:
using PyPlot
e₁ = [1,0,0,0,0]
e₅ = [0,0,0,0,1]
n = 1:400

figure(figsize=(12,3))

subplot(1,3,1)
plot(n, [e₅'*(M^n - M^(n-1))*e₁ for n in n], "bo-")
title(L"probability of reaching state $e_5$ in $n$ steps")
xlabel(L"number $n$ of steps")
ylabel("probability")

subplot(1,3,2)
semilogy(n, [e₅'*(M^n - M^(n-1))*e₁ for n in n], "bo-")
title("same plot, semilog scale")
xlabel(L"number $n$ of steps")

subplot(1,3,3)
plot(n, [(0.9864222278739748)^n / (e₅'*(M^n - M^(n-1))*e₁) for n in n], "bo-")
title("same plot, guessing scale")
xlabel(L"number $n$ of steps")

PyObject Text(0.5, 0, 'number $n$ of steps')

### (c)
Since $M = X\Lambda X^{-1}$, we know that $M^n = X\Lambda^nX^{-1}$.  
Substituting into the summation gives
$$\sum_{n=1}^\infty n (X\Lambda^nX^{-1} - X\Lambda^{n-1}X^{-1}) = X\left(\sum_{n=1}^\infty n(\Lambda^n - \Lambda^{n-1})\right)X^{-1} = X\left(\sum_{n=1}^\infty n\Lambda^n - \sum_{n=1}^\infty n\Lambda^{n-1}\right)X^{-1} = X\lim_{n\rightarrow \infty}\left(n\Lambda^n - \sum_{i = 0}^{n - 1}\Lambda^i\right)X^{-1}$$
If $\lambda = 1$, 
$$\lim_{n\rightarrow \infty}\left(n\lambda^n - \sum_{i = 0}^{n - 1}\lambda^i\right) = 0$$
Otherwise we have $|\mathrm{Re}(\lambda)| < 1$, 
$$\sum_{n=1}^\infty n\lambda^n - \sum_{n=1}^\infty n\lambda^{n-1} = \frac{\lambda}{(1 - \lambda)^2}\left(1 - \frac 1 {\lambda}\right) = \frac{1}{\lambda - 1}$$
Thus
$$\sum_{n=1}^\infty n (M^n - M^{n-1}) = X
\begin{pmatrix} 
    \frac{1}{\lambda_1 - 1} & & & & \\ 
    & \frac{1}{\lambda_2 - 1} & & & \\ 
    & & \frac{1}{\lambda_3 - 1} & & \\ 
    & & & \frac{1}{\lambda_4 - 1} & \\ 
    & & & & 0 
\end{pmatrix} X^{-1}$$

### (d)

In [3]:
# average number of steps to reach e₅, summed numerically:
@show sum([n * e₅'*(M^n-M^(n-1))*e₁ for n in 1:10^5])

function sumofsteps(p)
    if p ≈ 1
        return 0
    end
    return 1 / (p - 1)
end

sum(e₅'* X * Diagonal(sumofsteps.(λ)) * X^-1 * e₁)

sum([n * e₅' * (M ^ n - M ^ (n - 1)) * e₁ for n = 1:10 ^ 5]) = 75.11223892250649


75.11223892250644 + 7.317063621670172e-15im

## Problem 2 [5+5+5]

### (a)
Since we have 
$$M x_k = \lambda_k x_k$$
multiply both side by $o^T$ on the left,  
$$o^TM x_k = o^Tx_k = \lambda_k o^Tx_k \Rightarrow (1 - \lambda)o^Tx_k = 0$$
If $\lambda \neq 1$, then $o^Tx_k = 0$ and $o$ is orthogonal to $x_k$.  

### (b)

In [9]:
o = [1, 1, 1, 1, 1]
@show o' * X 

o' * X = ComplexF64[4.163336342344337e-16 - 0.0im -8.187894806610529e-16 - 4.163336342344337e-17im -8.187894806610529e-16 + 4.163336342344337e-17im 1.1102230246251565e-15 - 0.0im 1.0 - 0.0im]


1×5 adjoint(::Vector{ComplexF64}) with eltype ComplexF64:
 4.16334e-16-0.0im  …  1.11022e-15-0.0im  1.0-0.0im

### (c)
$$ x = c_1 x_1 + \cdots + c_m x_m $$
$$ o^Tx = c_m o^Tx_m $$
$$ c_m = \frac{o^Tx}{o^Tx_m}$$

## Problem 3 [4+4+4+4+4+4]

### (a)
Since we have
$$
\begin{cases}
    g_{k+2} & = (1 - w)g_{k+1} & + wg_k \\
    g_{k+1} & =        g_{k+1}
\end{cases}$$
Therefore
$$\begin{bmatrix} g_{k+2} \\ g_{k+1} \end{bmatrix} = \underbrace{\begin{bmatrix} 1 - w & w \\ 1 & 0 \end{bmatrix}}_{A}\begin{bmatrix} g_{k+1} \\ g_k \end{bmatrix}$$

### (b)
$$\det (A - \lambda I) = (1 - w - \lambda)(-\lambda) - w = \lambda^2 + (w - 1)\lambda - w = 0 \Rightarrow \lambda = \frac{-w + 1 \pm \sqrt{(w - 1)^2 + 4w}}{2} = \frac{-w + 1 \pm \sqrt{(w + 1)^2}}{2} = \frac{-w + 1 \pm (w + 1)}{2} = 1, -w$$

In [1]:
using LinearAlgebra
A(w) = [1-w w
       1 0]
λ, X = eigen(A(0.1))

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
2-element Vector{Float64}:
 -0.1
  1.0
vectors:
2×2 Matrix{Float64}:
 -0.0995037  0.707107
  0.995037   0.707107

### (c)
When $w$ is closer and closer to $-1$, the answer to (b) is still correct.  
However, when $w = -1$, $A = \begin{bmatrix} 2 & -1 \\ 1 & 0 \end{bmatrix}$ and there are only one eigenvalue $\lambda = 1$ with multiplicity of 2. Take a look at $A - I$,  
$$A - I = \begin{bmatrix} 1 & -1 \\ 1 & -1 \end{bmatrix}$$
The null space of that matrix only have dimension of $1$ (since it has a $1$-dimension row space), there is only $1$ eigenvector and $A$ is not diagonalizable under this circumstance.  

### (d)
It goes to a nonzero constant no matter what $x_0$ is because $|-w| < 1$ and there is another eigenvector with eigenvalue of $1$.  

### (e)
Let's find the eigenvectors first.  
$$A - I = \begin{bmatrix} - w & w \\ 1 & -1 \end{bmatrix}$$
The null space has a basis of $\begin{bmatrix} 1 \\ 1 \end{bmatrix}$.  
$$A + wI = \begin{bmatrix} 1 & w \\ 1 & w \end{bmatrix}$$
The null space has a basis of $\begin{bmatrix} -w \\ 1 \end{bmatrix}$.  

We have $Q = \begin{bmatrix}1 & -w \\ 1 & 1\end{bmatrix}$ and $\Lambda = \begin{bmatrix} 1 \\ & -w \end{bmatrix}$,  
Next we compute $Q^{-1}$ using the $2\times 2$ inversion formula.  
$$Q^{-1} = \frac 1 {\det Q}\begin{bmatrix}1 & w \\ -1 & 1\end{bmatrix} = \frac 1 {1 + w}\begin{bmatrix}1 & w \\ -1 & 1\end{bmatrix}$$

Finally, we can finish the limit:
$$\lim_{n\rightarrow \infty} A^n = \lim_{n\rightarrow \infty} Q \Lambda^n Q^{-1} = \begin{bmatrix}1 & -w \\ 1 & 1\end{bmatrix}\begin{bmatrix} 1 \\ & 0 \end{bmatrix}\frac 1 {1 + w}\begin{bmatrix}1 & w \\ -1 & 1\end{bmatrix} = \begin{bmatrix} 1 & 0 \\ 1 & 0 \end{bmatrix}\frac 1 {1 + w}\begin{bmatrix}1 & w \\ -1 & 1\end{bmatrix} = \frac 1 {1 + w}\begin{bmatrix}1 & w \\ 1 & w\end{bmatrix}$$

### (f)
We are going to find
$$\lim_{n\rightarrow \infty}A^nx_0$$
By direct substitution,  
$$\lim_{n\rightarrow \infty}A^nx_0 = \frac 1 {1 + 0.5}\begin{bmatrix}1 & 0.5 \\ 1 & 0.5\end{bmatrix}\begin{bmatrix}1 \\ 0\end{bmatrix} = \begin{bmatrix}\frac{1}{1.5} \\ \frac{1}{1.5}\end{bmatrix}$$
Therefore
$$\lim_{n\rightarrow \infty} g_n = \frac{1}{1.5} = \frac 2 3$$

In [9]:
An(w) = 1 / (1 + w) * [1 w; 1 w]
g = [1, 0]
w = 0.5
An(w) * g

2-element Vector{Float64}:
 0.6666666666666666
 0.6666666666666666